In [14]:
import codecs
import networkx as nx
#import numpy as np
import pandas as pd
import pickle
import re
from gensim.models import KeyedVectors
#from numpy.linalg import norm
from os import listdir
from os.path import isfile, join
from utils import printProgressBar

# column number
graph_1 = 3
graph_2 = 6
cat_1 = 8
cat_2 = 10
cstr_1 = 14
cstr_2 = 17
complexite = 19
orientation = 21
fichier_origine = 43

# Extract all valid cstr (Xeur, Xette, reX, ...)

In [ ]:
input_dir = 'families'
input_files = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]
valid_cstr = set()
counter = 0
for input_file in input_files:
    with codecs.open(join(input_dir, input_file), 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f):
            if line_num >= 2:
                elements = line.replace(' ', '').split('\t')
                valid_cstr.add(elements[cstr_1].replace('1', '').replace('2', ''))
                valid_cstr.add(elements[cstr_2].replace('1', '').replace('2', ''))
    counter += 1
    printProgressBar(counter, len(input_files), prefix='Progress:', suffix='complete', length=50, decimals=2)
valid_cstr.update(['dX', 'rX', 'réX'])
print(sorted(list(valid_cstr)))

# String comparison

In [ ]:
family_dict = dict()
with codecs.open('summary_of_families.txt', 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f):
        if line_num == 0:
            continue
        cols = line.split('\t')
        lexemes = cols[2].replace('{','').replace('}','').replace('\'','').split(', ')
        lexeme_set = set()
        for lexeme in lexemes:
            lexeme_set.add(lexeme.split('_')[0])
        family_dict[cols[0]] = lexeme_set
print(len(family_dict))

In [ ]:
family_dict_keys = list(family_dict.keys())
min_proportion = 0.5
df = pd.DataFrame(columns=['family_id_1', 'family_id_2', 'suspected', 'construction'])
for k1 in range(0, len(family_dict)):
    for k2 in range(k1+1, len(family_dict)):
        set1 = family_dict[family_dict_keys[k1]]
        set2 = family_dict[family_dict_keys[k2]]
        connected = False
        suspected = ''
        construction = ''
        for s1 in set1:
            for s2 in set2:
                if s1 in s2 and s2.replace(s1, 'X') in valid_cstr:
                    construction = s2.replace(s1, 'X')
                    connected = True
                    break
                elif s2 in s1 and s1.replace(s2, 'X') in valid_cstr:
                    construction = s1.replace(s2, 'X')
                    connected = True
                    break
            if connected:
                break
        if connected:
            df = df.append(pd.Series({
                'family_id_1': family_dict_keys[k1],
                'family_id_2': family_dict_keys[k2],
                'suspected': s1 + '-' + s2,
                'construction': construction
            }), ignore_index=True)
    printProgressBar(k1 + 1, len(family_dict), prefix = 'Progress:', suffix = 'complete', length = 50, decimals = 2)
df.to_excel('maybe_connected_cstr.xlsx', index=False)

# AOC-poset exploration

In [ ]:
context_with_header = pd.read_csv('context_from_maxgraph.csv', header=0, index_col=0)
family_ids = context_with_header.index

family_dict = dict()
with codecs.open('summary_of_families.txt', 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f):
        if line_num == 0:
            continue
        cols = line.split('\t')
        lexemes = cols[2].replace('{','').replace('}','').replace('\'','').split(', ')
        lexeme_set = set()
        for lexeme in lexemes:
            lexeme_set.add(lexeme.split('_')[0])
        family_dict[cols[0]] = lexeme_set
print(len(family_dict))

In [ ]:
L = nx.DiGraph()
vertex_graph_dict = dict()
with codecs.open(join('posets', 'families_simplified_maxgraph.dot'), 'r', encoding='utf-8') as f:
    for line in f:
        if '->' in line:  # a line showing edges between concepts
            elements = line.split()
            L.add_edge(vertex_graph_dict[elements[2]], vertex_graph_dict[elements[0]])
        elif 'shape' in line:  # a line describing a concept 
            vertex_number = line.split()[0]
            graph_number = re.search('Attribute (.*)\|', line.replace('\\n', '')).group(1)
            object_string = line.split('|')[2]
            objects = object_string.split('\\n')
            family_id_set = set()
            for obj in objects:
                if obj == '' or '}' in obj:
                    continue
                family_id_set.add(family_ids[int(obj.split()[1])])
            L.add_node(graph_number, families=family_id_set)
            vertex_graph_dict[vertex_number] = graph_number

In [ ]:
L.nodes['3800']['families']

In [ ]:
family_dict['F03800-10']

In [ ]:
number_of_maxgraphs = 3903
temp = codecs.open('temp.txt', 'w')
counter = 0
teetotal = 0
for g1 in range(0, number_of_maxgraphs):
    for g2 in range(g1 + 1, number_of_maxgraphs):
        if not (str(g1) in nx.descendants(L, str(g2)) or str(g2) in nx.descendants(L, str(g1))):
            # if a concept is not a descendant of another
            # temp.write(str(g1) + ' ' + str(g2) + '\n')
            teetotal += 1
        counter += 1
        printProgressBar(counter, number_of_maxgraphs * number_of_maxgraphs / 2, prefix = 'Progress:', suffix = 'complete', length = 50, decimals = 2)
temp.close()
print(teetotal)

# word2vec

In [5]:
model = KeyedVectors.load_word2vec_format('frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin', binary=True, unicode_errors="ignore")

In [27]:
model.most_similar('approchement')

KeyError: "Key 'approchement' not present"

In [29]:
model.similarity('boucher', 'déboucher')

0.0009172685